In [1]:
!pip install autogluon.tabular[all]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [7]:
from sklearn.model_selection import train_test_split
dataset = TabularDataset('adult.csv')

df_train, df_test = train_test_split(dataset, test_size=0.25)

Loaded data from: adult.csv | Columns = 15 / 15 | Rows = 32561 -> 32561


In [10]:
train_data = df_train
train_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
23363,30,Private,111567,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,48,United-States,<=50K
30809,43,Self-emp-not-inc,118261,Masters,14,Divorced,Other-service,Unmarried,White,Female,0,0,30,United-States,<=50K
9953,22,Private,290504,HS-grad,9,Never-married,Other-service,Other-relative,White,Male,0,0,40,United-States,<=50K
5736,27,Private,219371,HS-grad,9,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States,<=50K
24069,28,Private,214858,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K


In [11]:
label = 'income'
print(f"Unique classes: {list(train_data[label].unique())}")

Unique classes: ['<=50K', '>50K']


In [12]:
#4h training time
predictor_income = TabularPredictor(label=label, path="agModels_predict_income").fit(train_data, time_limit=14400, presets='best_quality')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.55 GB / 12.67 GB (83.3%)
Disk Space Avail:   190.69 GB / 225.83 GB (84.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stac

In [13]:
test_data = df_test
test_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
4698,31,Self-emp-inc,41493,Bachelors,13,Never-married,Farming-fishing,Not-in-family,White,Female,0,0,45,United-States,<=50K
26722,36,Self-emp-not-inc,67728,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,United-States,>50K
24623,63,Self-emp-not-inc,28612,HS-grad,9,Widowed,Sales,Not-in-family,White,Male,0,0,70,United-States,<=50K
28124,65,Private,83800,Some-college,10,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,27,United-States,<=50K
2049,49,Self-emp-not-inc,219718,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,15024,0,40,United-States,>50K


In [15]:
y_pred = predictor_income.predict(test_data)
y_pred.head()  # Predictions

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


,income
4698,<=50K
26722,>50K
24623,<=50K
28124,<=50K
2049,>50K


In [ ]:
predictor_income.evaluate(test_data)

{'accuracy': 0.8409253761899887,
 'balanced_accuracy': 0.7475663839529563,
 'mcc': 0.5345297121913682,
 'roc_auc': 0.884716037791454,
 'f1': 0.6296472831267874,
 'precision': 0.7034078807241747,
 'recall': 0.5698878343399483}

In [18]:
predictor_income.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.869058,0.876536,accuracy,1.103640,0.769734,31.040582,1.103640,0.769734,31.040582,1,True,4
1,WeightedEnsemble_L3,0.869058,0.876536,accuracy,1.105835,0.778260,31.776195,0.002196,0.008525,0.735613,3,True,8
2,WeightedEnsemble_L2,0.869058,0.876536,accuracy,1.106402,0.772812,31.406883,0.002762,0.003078,0.366302,2,True,6
3,LightGBMXT_BAG_L2,0.868567,0.876126,accuracy,5.210397,4.648569,108.939900,0.646262,0.400486,30.879231,2,True,7
4,LightGBMXT_BAG_L1,0.865618,0.871171,accuracy,2.483268,1.672700,34.939443,2.483268,1.672700,34.939443,1,True,3
5,RandomForestGini_BAG_L1,0.854563,0.857535,accuracy,0.808378,1.123140,11.923350,0.808378,1.123140,11.923350,1,True,5
6,KNeighborsUnif_BAG_L1,0.776686,0.773423,accuracy,0.079572,0.405211,0.091424,0.079572,0.405211,0.091424,1,True,1
7,KNeighborsDist_BAG_L1,0.769193,0.761384,accuracy,0.089277,0.277297,0.065870,0.089277,0.277297,0.065870,1,True,2


In [25]:
#leaderbaord download
import pandas as pd

leaderboard_df = predictor_income.leaderboard(test_data, extra_info=True)

print(leaderboard_df)

                     model  score_test  score_val eval_metric  pred_time_test  \
0          LightGBM_BAG_L1    0.869058   0.876536    accuracy        1.143174   
1      WeightedEnsemble_L2    0.869058   0.876536    accuracy        1.145506   
2      WeightedEnsemble_L3    0.869058   0.876536    accuracy        1.146225   
3        LightGBMXT_BAG_L2    0.868567   0.876126    accuracy        6.281249   
4        LightGBMXT_BAG_L1    0.865618   0.871171    accuracy        3.423796   
5  RandomForestGini_BAG_L1    0.854563   0.857535    accuracy        0.758113   
6    KNeighborsUnif_BAG_L1    0.776686   0.773423    accuracy        0.144385   
7    KNeighborsDist_BAG_L1    0.769193   0.761384    accuracy        0.162730   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.769734   31.040582                 1.143174                0.769734   
1       0.772812   31.406883                 0.002332                0.003078   
2       0.778260   31.77619

In [26]:
from google.colab import files

leaderboard_df.to_csv('autogluon_leaderboard_classification_normal.csv', index=False)

files.download('autogluon_leaderboard_classification_normal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
#predictor scores download
import pandas as pd
from google.colab import files

#predict score
evaluation_results = predictor_income.evaluate(test_data)

#to df
evaluation_df = pd.DataFrame([evaluation_results])

print(evaluation_df)

#to CSV
evaluation_df.to_csv('autogluon_classification_predictorScore_normal.csv', index=False)


files.download('autogluon_classification_predictorScore_normal.csv')

   accuracy  balanced_accuracy       mcc   roc_auc        f1  precision  \
0  0.869058           0.794944  0.626124  0.925805  0.706012   0.770156   

     recall  
0  0.651731  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>